<a href="https://colab.research.google.com/github/JairParra/NLPChallengeAI4Good/blob/master/final_results/Impact_Sentence_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()

In [0]:
!wget http://embeddings.net/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin

In [0]:
!pip install word2vec

In [99]:
import word2vec
model = word2vec.load('frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin')
indexes, scores = model.similar('intéressant')
model.generate_response(indexes,scores).tolist()

[('intéressante', 0.6818215686962036),
 ('instructif', 0.6677658999833322),
 ('assez', 0.630273572192353),
 ('intéressants', 0.6282307765327024),
 ('très', 0.6215654303816782),
 ('utile', 0.5970137708328055),
 ('rébarbatif', 0.591519468943563),
 ('intéressantes', 0.5864250462172865),
 ('judicieux', 0.5620384868557253),
 ('beaucoup', 0.5607017562920658)]

In [0]:
import pandas as pd
sentences_augmented = pd.read_csv("sentences_classified_augmented.csv")

In [156]:
import nltk
from nltk import word_tokenize

# Donwload punctuation
nltk.download('punkt')

import re
regexp = re.compile(r'((([\wÀ-ÿ])*(.){0,1}\s+)){0,10}(\d+([.]{0,3}\d+)*)((([\wÀ-ÿ])*(.){0,1}\s+)){0,10}.')
word_bank = set(['individuel','individuelle','personnes','femmes','réfugiés', 'participant','participants','moyenne', 'augmentation', 'diminution', 'plus', 'moins', 'impact', 'lamoyenne', 'moyennne', 'moyennede', 'moyenn', 'hausse', 'baisse', 'accroissement', 'uneaugmentation', 'encore', 'très', 'suffisament', 'beaucoup', 'assez', 'excessivement', 'relativement' ])
has_impact_fixed = []


def has_words_from_bank(sentence):
  split = set(word_tokenize(sentence, language='french'))
  return len(split.intersection(word_bank)) > 0

  
for row in sentences_augmented.itertuples():
  if has_words_from_bank(row.sentence_RESULTATS_2018):
    has_impact_fixed_item = "yes"
  elif regexp.match(row.sentence_RESULTATS_2018) and row[3] == "no" and row[2] != "Autre":
    has_impact_fixed_item = "yes"
  elif row[4] == "yes":
    has_impact_fixed_item = "yes"
  else:
    has_impact_fixed_item = "no"
  has_impact_fixed.append(has_impact_fixed_item)

sentences_augmented = sentences_augmented.assign(has_impact_fixed = has_impact_fixed)

sentences_augmented.to_csv("sentences_augmented_has_impact_fixed.csv")

files.download("sentences_augmented_has_impact_fixed.csv")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:

from sklearn.feature_extraction.text import TfidfVectorizer

def get_sentence_embedding_tfidf(sentence, vectorizer, w2v_model):
  tf_idf = vectorizer.transform([sentence])
  tfidf_map = dict(zip([vectorizer.get_feature_names()[idx] for idx in tf_idf.indices], tf_idf.data))
  sentence_embedding = []
  for word in word_tokenize(sentence, language='french'):
    lw = word.lower()
    if lw in w2v_model:
      word_embedding = w2v_model[lw]
      if lw in tfidf_map:
        w = tfidf_map[lw]
      else:
        w = 0.0000000001
      word_embedding *= w
      sentence_embedding.append(word_embedding)
  
  if len(sentence_embedding) > 0:
    se_avg = sentence_embedding[0]
    if len(sentence_embedding) > 1:
      for se in sentence_embedding[1:]:
        se_avg += se
      se_avg /= len(sentence_embedding)
  else: 
    se_avg = [0.0 for i in range(0,200)]
  return se_avg

corpus = sentences_augmented["sentence_RESULTATS_2018"].to_list()

vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

import numpy as np

sentences_embedding = []

for row in sentences_augmented.itertuples():
  se_avg = get_sentence_embedding_tfidf(row.sentence_RESULTATS_2018, vectorizer, model)
  sentences_embedding.append(se_avg)


In [0]:
X = sentences_embedding
# y = sentences_augmented["Has impact"].to_list()
y = has_impact_fixed
y = list(map(lambda arg: 1.0 if arg=='yes' else 0.0, y))
sample_weight = list(map(lambda arg: 1.0 if arg == 1.0 else 0.4181, y))

In [247]:
from sklearn import svm

clf = svm.SVC(gamma='scale')
clf.fit(X, y, sample_weight=sample_weight)  


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [248]:
clf.score(X,y)

0.3208868144690782

In [249]:
from sklearn.metrics import f1_score
y_pred = clf.predict(X)
# import random as rnd
# y_pred = [rnd.randint(0,1) for i in range(0,1714)]
f1_score(y, y_pred)


0.4616096207215542

In [96]:
print(y_pred)

[1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 

In [150]:
print(error)

[0. 0. 0. ... 0. 0. 0.]


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
print(vectorizer.get_feature_names())

tf_idf = vectorizer.transform(['And this is the third one.'])

test = 'And this is the third one.'.split()

for word in test:
  vectorizer

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [0]:
print(test)

['And', 'this', 'is', 'the', 'third', 'one.']


In [0]:
tfidf_map

scipy.sparse.csr.csr_matrix